# Bildcrpsprogram

## Problemdarstellung
Die Bilder aus den Mikroscopen sind sehr aufwendig zu erstellen. Für einen guten und umfangreichen Trainingsdatensatz sind jedoch sehr viele Aufnahmen erforderlich. Dazu können die großen Bilder in kleinere zerlegt werden. Hierbei ist wichtig das die Größe des Bildes in mm bekannt ist und mit den neuen Bildern abgespeichert wird.

## Anforderungen 
- Ein Bild muss eingelesen werden
- Größe dieses Bildes bestimmen
  - Größe eines Pixels mittels dem Refenecebalkens muss bestimmt werden
    - Entweder wird dies im vorraus bestimmt und im Dateinamen gespeichert, welcher dann einheitlich sein muss
    - Oder inerhalb des Programmes
    - > Es stellt sich heraus, dass der Refeencbalken immer an der Gleichen Stelle im Bild positioniert ist. Ist das Bild im Allgemeinen kleiner so ist der Balken entsprechend größer, ist das Bild größer, so ist der Balken kleiner
- Anzahl der neuen Bilder festlegen, die erstellt werden sollen
- Bilder in einem Zielordner abspeichern
  - checken ob der Zielordner schon erstellt ist, ansonsten einen erstellen
  - Pfad des Zielordners bestimmen


### Bild einlesen

In [1]:
from PIL import Image
import os

# Pfad zum Ordner mit den Bildern
img_folder_path = '/Users/fabi/Pictures/Bilder_Projektarbeit'
# Pfad zu den zugeschnittenen Bildern
crp_img_save_path = '/Users/fabi/Pictures/Bilder_Projektarbeit/Zugeschnitte_Bilder'

# Erstelle den Ordner für die zugeschnittenen Bilder, falls noch nicht vorhanden
if not os.path.exists(crp_img_save_path):
    os.mkdir(crp_img_save_path)
    print(f'Ordner "{crp_img_save_path}" wurde erstellt.')
else:
    print(f'Ordner "{crp_img_save_path}" ist vorhanden.')

# Erstelle den Ordner für die  Bilder, falls noch nicht vorhanden
if not os.path.exists(img_folder_path):
    os.mkdir(img_folder_path)
    print(f'Ordner "{img_folder_path}" wurde erstellt.')
else:
    print(f'Ordner "{img_folder_path}" ist vorhanden.')



#Länge des Referenzebalkens
referenzbalken_pixel = 1462  # Größe des Referenzbalkens in Pixeln
referenzbalken_mm = 2  # Größe des Referenzbalkens in mm

# Bildgröße
crp_img_width = 500
crp_img_height  = 500

crp_size = True

# Anzahl der Biilder pro Achse bestimmen bestimmen
anzahl_bilder_x = 12
anzahl_bilder_y = 12

anzahl = False

# Definieren des Schwellenwerts
threshold_edge = 200

Ordner "/Users/fabi/Pictures/Bilder_Projektarbeit/Zugeschnitte_Bilder" ist vorhanden.
Ordner "/Users/fabi/Pictures/Bilder_Projektarbeit" ist vorhanden.


In [2]:
import math
import cv2
import os
import numpy as np

# Durchsuche den Ordner nach Dateien
for filename in os.listdir(img_folder_path):
    # Überprüfen, ob die Datei eine Bilddatei ist (z. B. .jpg oder .png)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Voller Pfad zur Bilddatei
        image_path = os.path.join(img_folder_path, filename)
        
        # Öffne das Bild mit cv2 und konvertiere es in Graustufen
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Bestimme die Größe des Bildes in mm (Annahme: Größe des Referenzbalkens ist bekannt)
        # Beispielhaft wird hier die Größe als (Breite, Höhe) in mm ausgegeben
        breite_mm = img.shape[1] * referenzbalken_mm / referenzbalken_pixel
        höhe_mm = img.shape[0] * referenzbalken_mm / referenzbalken_pixel
        
        # Ausgabe der Bildgröße
        print(f"Datei: {filename} | Breite: {breite_mm} mm | Höhe: {höhe_mm} mm")

        crp_breite = crp_img_width
        crp_hoehe  = crp_img_height

        # Anzahl der Bilder aus der Bildgröße bestimmen
        anzahl_bilder_x = img.shape[1] // crp_img_width
        anzahl_bilder_y = img.shape[0] // crp_img_height

        # Rand berechnen der auf beiden Seiten freigehalten werden soll, damit alle Bilder mittig reinpassen
        rand_height = (img.shape[0] % crp_img_height) // 2
        rand_width  = (img.shape[1] % crp_img_width) // 2

        # Schleife für jeden Zuschneidebereich
        for x in range(anzahl_bilder_x):
            for y in range(anzahl_bilder_y):
                # Berechne die Koordinaten des aktuellen Zuschneidebereichs
                crp_x = (x * int(crp_breite)) + rand_width
                crp_y = (y * int(crp_hoehe) ) + rand_height
                
                # Dateiname für den zugeschnittenen Bereich
                crp_name = f"crp_x_{crp_x}_y_{crp_y}" 

                # Schneide das Bild entsprechend dem Zuschneidebereich zu
                crp_img = img[crp_y:crp_y+crp_hoehe, crp_x:crp_x+crp_breite]

                # Anzeigen des Bildes
                cv2.imshow("Zugeschnittenes Bild", crp_img)
                cv2.waitKey(0)

                # Mittelwerte der Außenkanten berechnen
                mean_top = np.mean(crp_img[0, :])
                mean_bottom = np.mean(crp_img[crp_hoehe - 1 , :])
                mean_right = np.mean(crp_img[:, crp_img_width - 1])
                mean_left = np.mean(crp_img[:,0])
                
                # Im zugeschnittenen Bild werden die Kanten mit einem Threshold verglichen um fest
                if mean_top > threshold_edge or mean_bottom> threshold_edge or mean_right > threshold_edge or mean_left > threshold_edge:
                    # Speichere den zugeschnittenen Bereich ab
                    crp_img.save(os.path.join(crp_img_save_path, crp_name + "threshold" + ".png"))
                else:
                    # Speichere den zugeschnittenen Bereich ab
                    crp_img.save(os.path.join(crp_img_save_path, crp_name + ".png"))

Datei: Probe_006_klein.png | Breite: 13.079343365253077 mm | Höhe: 5.879616963064295 mm


2023-05-02 09:32:06.310 Python[17678:1474049] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/org.python.python.savedState


In [22]:
import math
from PIL import Image
import os
from skimage import color
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

# Durchsuche den Ordner nach Dateien
for filename in os.listdir(img_folder_path):
    # Überprüfen, ob die Datei eine Bilddatei ist (z. B. .jpg oder .png)
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Voller Pfad zur Bilddatei
        image_path = os.path.join(img_folder_path, filename)
        
        # Öffne das Bild mit PIL
        with Image.open(image_path).convert('L') as img:
            # Bestimme die Größe des Bildes in mm (Annahme: Größe des Referenzbalkens ist bekannt)
            # Beispielhaft wird hier die Größe als (Breite, Höhe) in mm ausgegeben
            breite_mm = img.width * referenzbalken_mm / referenzbalken_pixel
            höhe_mm = img.height * referenzbalken_mm / referenzbalken_pixel
            
            # Ausgabe der Bildgröße
            print(f"Datei: {filename} | Breite: {breite_mm} mm | Höhe: {höhe_mm} mm")

            crp_breite = crp_img_width
            crp_hoehe  = crp_img_height

            # Anzahl der Bilder aus der Bildgröße bestimmen
            anzahl_bilder_x = img.width  // crp_img_width
            anzahl_bilder_y = img.height // crp_img_height

            # Rand berechnen der auf beiden Seiten freigehalten werden soll, damit alle Bilder mittig reinpassen
            rand_height = (img.height % crp_img_height) // 2
            rand_width  = (img.width % crp_img_width) // 2

            # Schleife für jeden Zuschneidebereich
            for x in range(anzahl_bilder_x):
                for y in range(anzahl_bilder_y):
                    # Berechne die Koordinaten des aktuellen Zuschneidebereichs
                    crp_x = (x * int(crp_breite)) + rand_width
                    crp_y = (y * int(crp_hoehe) ) + rand_height
                    
                    # Dateiname für den zugeschnittenen Bereich
                    crp_name = f"crp_x_{crp_x}_y_{crp_y}" 

                    # Schneide das Bild entsprechend dem Zuschneidebereich zu
                    crp_img = img.crop((crp_x, crp_y, crp_x + crp_breite, crp_y + crp_hoehe))

                    # Zugeschnittenes Bild in MatPltLib einladen um die Pixelwerte mit Numpy vergleichen zu können. Mit PIL ist der vergleich schwierirger 
                    np_crp_img = np.array(crp_img)
                    
                    # Anzeigen des Bildes
                    plt.imshow(np_crp_img, cmap= 'gray')
                    plt.show()

                    # Mittelwerte der Außenkanten berechnen
                    mean_top = np.mean(np_crp_img[0, :])
                    mean_bottom = np.mean(np_crp_img[crp_hoehe - 1 , :])
                    mean_right = np.mean(np_crp_img[:, crp_img_width - 1])
                    mean_left = np.mean(np_crp_img[:,0])
                    
                    # Im zugeschnittenen Bild werden die Kanten mit einem Threshold verglichen um festzustellen, ob es sich um ein Randbild handelt oder nicht
                    if mean_top > threshold_edge or mean_bottom> threshold_edge or mean_right > threshold_edge or mean_left > threshold_edge:
                        # Speichere den zugeschnittenen Bereich ab
                        crp_img.save(os.path.join(crp_img_save_path, crp_name + "threshold" + ".png"))
                    else:
                        # Speichere den zugeschnittenen Bereich ab
                        crp_img.save(os.path.join(crp_img_save_path, crp_name + ".png"))


            
            
            


ValueError: the input array must have size 3 along `channel_axis`, got (4298, 9561)

## Bestimmung der Größe des Messbalkens
Die Bilder werden in GIMP gelanden und mit dem Messtool vermessen. Es wird geprüft ob der Maßstabsbalken immer an der selben Stelle und immer die gleiche größe hat. Somit kann die Größe eines Pixels mit der Bildgröße ermittelt werden.
### Messergebnisse
h = Höhe

l = Länge

p =  Positions des unteren linken Ecks des Messbalken
#### Probe_006_klein.png
h = 38

l = 1462

p = 9454 x 3905

g = 9561 x 4298
#### Probe_006_quadratisch.png
h = 49

l = 1462

p = 7547 x 7980

g = 7683 x 8485
#### Probe_007_rechteckig.png
h = 56

l = 1462

p = 13132 x 9280

g = 13291 x 9867
## Erkenntniss
Der Maßstabsbalken ist immer 1462 Pixel lang

## Bilder zuschneiden

Die Bilder werden zugeschnitten und durchnummeriert in einem neunen Ordner gespeichert

# Bilder zuschneiden
            if anzahl is True:
                # Breite und Höhe der crpe bestimmen
                crp_breite = img.width / anzahl_bilder_x
                crp_hoehe = img.height / anzahl_bilder_y
                # Schleife für jeden Zuschneidebereich
                for x in range(anzahl_bilder_x):
                    for y in range(anzahl_bilder_y):
                        # Berechne die Koordinaten des aktuellen Zuschneidebereichs
                        crp_x = x * int(crp_breite)
                        crp_y = y * int(crp_hoehe)
                        crp_name = f"crp_x{crp_x}_y{crp_y}.jpg" # Dateiname für den zugeschnittenen Bereich
                        
                        # Schneide das Bild entsprechend dem Zuschneidebereich zu
                        crp_img = img.crop((crp_x, crp_y, crp_x + crp_breite, crp_y + crp_hoehe))

                        # Füge den crps-Idize hinzu
                        crp_idize = f"Zuschnitt x{x} y{y} - Größe: {crp_breite} x {crp_hoehe} - Rand: Nein"
                        
                        # Speichere den zugeschnittenen Bereich ab
                        crp_img.save(os.path.join(crp_img_save_path, crp_name))
